In [1]:
import os
import openai
import tiktoken
# from dotenv import load_dotenv
import time
import re
# from google.colab import drive
# drive.mount('/content/gdrive')
# path = '/content/gdrive/My Drive/585data/'

In [2]:
# path = '../data/'    # change the path as needed
path = '../' #'/content/gdrive/My Drive/585data/'

def read_data(file):
    with open (path+file) as t:
        data = t.readlines()
    return data

# ChatGPT API only allow 3 prompts per minute, so we cannot make 3000 as other models here.
test_set = read_data('test_data.txt')[:3000]

# type(train_set)
print(test_set[:2], '\n', len(test_set))

['{"groundTruth": ["现身说法"], "candidates": [["旷日持久", "公正廉洁", "苦口婆心", "现身说法", "白日做梦", "深入浅出", "肺腑之言"]], "content": "只要路过的旅客稍有迟疑，或者对他们的宣传单多看几眼，基本上这个旅客就别想轻松脱身了，记者就在9月3日接站时目睹了这样一幕：一个学生接过招生人员递来的宣传单，只是问了一下“你们学校有没有分数要求？”两个招生人员就“白话”开了，一个表示分数都好说，只要有好学的精神；另一个则#idiom#，大讲自己选择的专业现在收获颇丰；最后在招生人员“我们学校毕业后可以完全解决就业”的忽悠下，这个学生旅客被他们拉上了到校参观的班车。", "realCount": 1}\n', '{"groundTruth": ["神来之笔", "赞不绝口"], "candidates": [["画龙点睛", "悔过自新", "拍案叫绝", "鬼斧神工", "神来之笔", "颠倒黑白", "中流砥柱"], ["敬谢不敏", "拍案叫绝", "心悦诚服", "叹为观止", "赞不绝口", "口口声声", "扬眉吐气"]], "content": "亨利的这个#idiom#被法国媒体形容为“空中舞蹈”，亨利自己对球队表现也很满意，“上半场开局一般，但很快觉醒，下半场的进攻让人看到真正的法国，尤其是我们的速度让对方有了麻烦。”而和亨利搭档的本泽马对老大哥#idiom#，“和他在一起配合很容易，他给了我很多信心。”", "realCount": 2}\n'] 
 3000


In [3]:
# load_dotenv()

api_path = path + 'openai_gpt_api_key.txt'

### read api key from file
with open(api_path, "r") as file:
    API_KEY = file.read().strip()

openai.api_key = API_KEY
enc = tiktoken.get_encoding("cl100k_base")

messages = []

def ask_gpt(input_text, print_results = True, delay: float = 1):
    """
    API call to gpt-3.5-turbo. Note: each time the function is called, the API call will keep previous calls to the model as context until the kernel is restarted.
    If you would like to reset the model's conversational context, restart kernel and run the function again.

    Parameters:
    ------
    `input`: str
        user input that is passed to model. Used in the same way information is passed to the model in the OpenAI web interface for ChatGPT
    `print_results`: bool = True
        if model's returned text exceeds a certain number of characters, it includes new line marking "\n". if print_result == True,
        function splits printout by "\n" and print results out line by line, otherwise results are only returned as a list object,
        with each list item as a line of the printout
    `delay`: float
        amount by which to delay sequential API calls (calls in sequence that are too frequent can lead to a ban in certain instances - this avoids that)
    
    Returns:
    ------
    `result_list`: list
        only returned if `print_result` == False. Return is a list object with returned text from the model split by "\n".
        If returned text is short enough and no text is present, list will only be length of 1
    """
    messages = []
    
    time.sleep(delay)
    
    # global messages
    messages.append({"role": "user", "content": input_text})
    chat_test = openai.ChatCompletion.create(
        model = "gpt-3.5-turbo",
        temperature = 0.1,
        messages = messages)
    
    result_list = chat_test.choices[0].message.content#.split("\n") # if not split, the returned text is more than one line, the printout has "\n" in it

    # if print_results == True:
    #     for line in result_list:
    #         print (line)
    # else:
    return result_list


In [4]:
def prompt(data):
    '''data shall be the output of `read_data`'''
    text_input = []
    gold = []
    for i in range(len(data)):
        data[i] = eval(data[i])
        input_text = data[i]['content']
        candidates = data[i]['candidates']
        ground_truth = set(data[i]['groundTruth'])
        gold.append(ground_truth)

        candidate_str = ''
        for candidate in candidates:
            candidate_str += '('+'|'.join(candidate)+')'
        
        # preprocess_idx = -1
        # def replace(match):
        #     nonlocal preprocess_idx
        #     preprocess_idx += 1
        #     return 'extra{}'.format(preprocess_idx)
        # input_text = re.sub(r'#idiom#', replace, input_text)

        instruction = f'''示例1：(意气风发|街谈巷议|人才辈出|一脉相传|后继有人|发扬光大|腥风血雨)(平易近人|落落大方|八仙过海|彬彬有礼|史无前例|盛气凌人|好自为之)(不拘小节|风流潇洒|无病呻吟|言谈举止|壮志凌云|关门闭户|温文尔雅)
历史上的蒋南翔是著名的“一二九”学生救亡运动的领导人之一，他在清华校长之位14年期间，不但很好的继承了清华建校之初的优秀传统与理念，而且更加的#idiom#，他把清华的教师队伍扩大了将近5倍，将清华本科人数破万，为新中国培养了大量的有用人才。在《天行健》中饰演蒋南翔的刘威是观众所熟悉的著名实力派演员，早在1987年刘威就在《关东大侠》中饰演豪爽仗义的关云天一角而获得了金鸡奖最佳男主角的提名，后来更是因在《唐明皇》中精湛的表演而一举夺得金鹰奖最佳男演员奖。此次《天行健》选定刘威来出演正是看中了他#idiom#的表演方式和对人物深入内心的刻画。至此，《天行健》中涉及的三位清华校长的人选都已经曝光，#idiom#的第一任校长赵文?、稳重坚毅的第二任校长孙逊、亲切务实的第三任校长刘威，再加上梁思成、林徽因、朱自清、闻一多等一批“大师”的加盟，相信作为清华百年校庆重点项目之一的《天行健》一定会带领观众重温那段不能抹去的历史。 
示例1的正确答案为：发扬光大,平易近人,温文尔雅
示例2：(深恶痛绝|人人自危|恨入骨髓|不胜枚举|嗤之以鼻|走马看花|不屑一顾)
另据了解，北京一个对垃圾短信#idiom#的老人，利用该软件总共呼死了近2000个号码。20分钟呼上万号码记者昨天在百度里输入“呼死你软件”，出现了7000多个相关网页，随机登录几个网站，发现软件均需花钱购买，价格从200元至500元不等。 
示例2的正确答案为：深恶痛绝
请按照上述示例，从下列括号中分别选择合适的成语填入"#idiom#"处，请仅回复成语，不要回复其他字符：{candidate_str}'''
        
        text_input.append(instruction+'\n'+input_text)

    return text_input, gold

In [5]:
test_input, gold = prompt(test_set)

In [6]:
test_input[1]

'示例1：(意气风发|街谈巷议|人才辈出|一脉相传|后继有人|发扬光大|腥风血雨)(平易近人|落落大方|八仙过海|彬彬有礼|史无前例|盛气凌人|好自为之)(不拘小节|风流潇洒|无病呻吟|言谈举止|壮志凌云|关门闭户|温文尔雅)\n历史上的蒋南翔是著名的“一二九”学生救亡运动的领导人之一，他在清华校长之位14年期间，不但很好的继承了清华建校之初的优秀传统与理念，而且更加的#idiom#，他把清华的教师队伍扩大了将近5倍，将清华本科人数破万，为新中国培养了大量的有用人才。在《天行健》中饰演蒋南翔的刘威是观众所熟悉的著名实力派演员，早在1987年刘威就在《关东大侠》中饰演豪爽仗义的关云天一角而获得了金鸡奖最佳男主角的提名，后来更是因在《唐明皇》中精湛的表演而一举夺得金鹰奖最佳男演员奖。此次《天行健》选定刘威来出演正是看中了他#idiom#的表演方式和对人物深入内心的刻画。至此，《天行健》中涉及的三位清华校长的人选都已经曝光，#idiom#的第一任校长赵文?、稳重坚毅的第二任校长孙逊、亲切务实的第三任校长刘威，再加上梁思成、林徽因、朱自清、闻一多等一批“大师”的加盟，相信作为清华百年校庆重点项目之一的《天行健》一定会带领观众重温那段不能抹去的历史。 \n示例1的正确答案为：发扬光大,平易近人,温文尔雅\n示例2：(深恶痛绝|人人自危|恨入骨髓|不胜枚举|嗤之以鼻|走马看花|不屑一顾)\n另据了解，北京一个对垃圾短信#idiom#的老人，利用该软件总共呼死了近2000个号码。20分钟呼上万号码记者昨天在百度里输入“呼死你软件”，出现了7000多个相关网页，随机登录几个网站，发现软件均需花钱购买，价格从200元至500元不等。 \n示例2的正确答案为：深恶痛绝\n请按照上述示例，从下列括号中分别选择合适的成语填入"#idiom#"处，请仅回复成语，不要回复其他字符：(画龙点睛|悔过自新|拍案叫绝|鬼斧神工|神来之笔|颠倒黑白|中流砥柱)(敬谢不敏|拍案叫绝|心悦诚服|叹为观止|赞不绝口|口口声声|扬眉吐气)\n亨利的这个#idiom#被法国媒体形容为“空中舞蹈”，亨利自己对球队表现也很满意，“上半场开局一般，但很快觉醒，下半场的进攻让人看到真正的法国，尤其是我们的速度让对方有了麻烦。”而和亨利搭档的本泽马对老大哥#idiom#，“和他在一起配合很容易，他给了我很多信心。”'

In [7]:
gold[:2]

[{'现身说法'}, {'神来之笔', '赞不绝口'}]

In [8]:
ask_gpt(test_input[1], delay = 0.1)

'神来之笔，拍案叫绝'

In [9]:
sys = []
for q in test_input:
    a = ask_gpt(q, delay=0.1)
    a = a.strip(' 。，答案：#').split(',')
    sys.append(set(a))
### The fee was used up so this is interrupted... 
### It took around 2 hours so we decided not to continue TAT
### We have run 2096 test data for now (2096/3000)

RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 0e042be6145339bcfc91680c4443d6bc in your message.)

In [11]:
len(sys)

2069

In [12]:
with open('chatgpt_ans.txt', 'w') as a:
    for s in sys:
        a.write(str(s)+'\n')    

In [13]:
def f1_score(sys, gold):
    tp = 0
    total = 0
    pos = 0
    for s, g in zip(sys, gold):
        total += len(g)
        pos += len(s)
        tp += len(g & s)
    precision = tp / pos if pos != 0 else 0
    recall = tp / total if total != 0 else 0
    f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) != 0 else 0
    return precision, recall, f1, tp

def accuracy(sys, gold, tp):
    total = 0
    for s, g in zip(sys, gold):
        total += len(g)
    return tp / total

In [14]:
p, r, f1, tp = f1_score(sys, gold)
print('ChatGPT: ')
print(f"Accuracy for test set is {accuracy(sys, gold, tp)}")
print(f"F1 score for test set is {f1}")

ChatGPT: 
Accuracy for test set is 0.18822124950258656
F1 score for test set is 0.20348462034846204


In [15]:
p = 0
for s, g in zip(sys, gold):
    if s != g:
        print(s, g)
        p += 1
    if p > 60:
        break

{'苦口婆心'} {'现身说法'}
{'亨利的这个神来之笔被法国媒体形容为“空中舞蹈”，亨利自己对球队表现也很满意，“上半场开局一般，但很快觉醒，下半场的进攻让人看到真正的法国，尤其是我们的速度让对方有了麻烦。”而和亨利搭档的本泽马对老大哥拍案叫绝，“和他在一起配合很容易，他给了我很多信心。”'} {'神来之笔', '赞不绝口'}
{'劳燕分飞'} {'难分难舍'}
{'强弩之末'} {'凶多吉少'}
{'示例答案：孤注一掷'} {'孤注一掷'}
{'墨守成规'} {'闭门造车'}
{'恰如其分'} {'锦上添花'}
{'目前，进行电子阅读的用户均有手机上网包月套餐，并不担心上网产生额外的流量费，也就是说其阅读的电子书基本是免费资源，这或许可以解释为什么电子阅读发展能够如此迅速。免费对消费者具有足够的吸引力：“零价格已经不仅仅是另一个价格了，它已经成为了一个让情感繁文缛礼的按钮，成为非理性快乐的源泉”。消费者购买再便宜的产品也会害怕吃亏，这种本能极大影响了其购买决策；而对于免费产品则根本不存在吃亏的问题，只有好处的现实会让其感情迅速充电，毫不犹豫地选择免费产品。所以坚持几年之内不赚钱而专心培育市场是开启电子“悦读”时代所必须的'} {'一触即发'}
{'分工合作，泰然自若，惊惶失措'} {'全神贯注', '惊惶失措'}
{'一般无二'} {'本来面目'}
{'字斟句酌'} {'洋洋洒洒'}
{'怒火中烧，自力更生'} {'自力更生', '气势汹汹'}
{'记者在现场看到，事故现场的沙堆积如山。沙场工作人员黄先生介绍说，当时沙堆又多又高，只要有一点震动，上面大量的沙子就会往下流动，大家都知道很危险，平时都不敢随便靠近的。事发当天上午9点左右，该女老板还问过他沙子质量如何，没想到10点多就发生意外了'} {'堆积如山'}
{'骑虎难下'} {'狗急跳墙'}
{'瞒天过海'} {'冒名顶替'}
{'这种感觉很快便让他恢复了坦然自若的感觉'} {'坦然自若'}
{'从政后，两人的成就皆令人刮目相待，爱德华通晓英国内政事务，曾帮助布朗起草政府年度工作草案。在外交舞台上，当过外交大臣的哥哥更有优势，美国国务卿希拉里就直率地表示戴维·米利班德给她留下了很好的印象'} {'刮目相待'}
{'力排众议，一成不变'} {'周而复始'}
{'这个场面有些滑稽，猛冲猛打人家根本不怕你，彼此

2069